<a href="https://colab.research.google.com/github/alishiabatool20-droid/Pearls-AQI-Predictor/blob/main/Pearls%20AQI%20%20Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests pandas


In [ ]:
import requests
import pandas as pd

API_KEY = "33495808f225717594d6c59635b63f87"
lat = 24.8607   # Karachi latitude
lon = 67.0011   # Karachi longitude
url_weather = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_KEY}"

response_weather = requests.get(url_weather)
data_weather = response_weather.json()

# convert json to dataframe
df_weather = pd.json_normalize(data_weather['list'])
print(df_weather.head())



           dt                                            weather  visibility  \
0  1758877200  [{'id': 803, 'main': 'Clouds', 'description': ...       10000   
1  1758888000  [{'id': 803, 'main': 'Clouds', 'description': ...       10000   
2  1758898800  [{'id': 802, 'main': 'Clouds', 'description': ...       10000   
3  1758909600  [{'id': 801, 'main': 'Clouds', 'description': ...       10000   
4  1758920400  [{'id': 800, 'main': 'Clear', 'description': '...       10000   

   pop               dt_txt  main.temp  main.feels_like  main.temp_min  \
0    0  2025-09-26 09:00:00     305.22           308.94         302.73   
1    0  2025-09-26 12:00:00     304.19           308.08         302.12   
2    0  2025-09-26 15:00:00     302.38           306.19         300.96   
3    0  2025-09-26 18:00:00     300.33           303.19         300.33   
4    0  2025-09-26 21:00:00     299.73           299.73         299.73   

   main.temp_max  main.pressure  main.sea_level  main.grnd_level  \
0     

In [ ]:
url_aqi = f"http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={API_KEY}"

response_aqi = requests.get(url_aqi)
data_aqi = response_aqi.json()

# convert to dataframe
df_aqi = pd.json_normalize(data_aqi['list'])
print(df_aqi.head())



           dt  main.aqi  components.co  components.no  components.no2  \
0  1758873600         2          91.71           0.01            0.03   
1  1758877200         2          91.36           0.01            0.03   
2  1758880800         2          91.26           0.01            0.03   
3  1758884400         2          91.01           0.01            0.03   
4  1758888000         2          90.67           0.00            0.03   

   components.o3  components.so2  components.pm2_5  components.pm10  \
0          47.54            0.09              8.34            34.41   
1          46.58            0.09              8.32            34.00   
2          45.90            0.09              8.32            33.73   
3          45.22            0.08              8.24            33.15   
4          44.76            0.08              8.04            32.00   

   components.nh3  
0               0  
1               0  
2               0  
3               0  
4               0  


In [ ]:
# Convert AQI timestamp to datetime
df_aqi['dt'] = pd.to_datetime(df_aqi['dt'], unit='s')

# Weather datetime
df_weather['dt'] = pd.to_datetime(df_weather['dt'], unit='s', errors='ignore')

# merge dono
df = pd.merge(df_weather, df_aqi, on='dt', how='inner')
print(df.head())


                   dt                                            weather  \
0 2025-09-26 09:00:00  [{'id': 803, 'main': 'Clouds', 'description': ...   
1 2025-09-26 12:00:00  [{'id': 803, 'main': 'Clouds', 'description': ...   
2 2025-09-26 15:00:00  [{'id': 802, 'main': 'Clouds', 'description': ...   
3 2025-09-26 18:00:00  [{'id': 801, 'main': 'Clouds', 'description': ...   
4 2025-09-26 21:00:00  [{'id': 800, 'main': 'Clear', 'description': '...   

   visibility  pop               dt_txt  main.temp  main.feels_like  \
0       10000    0  2025-09-26 09:00:00     305.22           308.94   
1       10000    0  2025-09-26 12:00:00     304.19           308.08   
2       10000    0  2025-09-26 15:00:00     302.38           306.19   
3       10000    0  2025-09-26 18:00:00     300.33           303.19   
4       10000    0  2025-09-26 21:00:00     299.73           299.73   

   main.temp_min  main.temp_max  main.pressure  ...  sys.pod  main.aqi  \
0         302.73         305.22           

/tmp/ipython-input-2255834805.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_weather['dt'] = pd.to_datetime(df_weather['dt'], unit='s', errors='ignore')


In [ ]:
# time-based features
df['hour'] = df['dt'].dt.hour
df['day'] = df['dt'].dt.day
df['month'] = df['dt'].dt.month

# AQI change rate
df['aqi_change'] = df['main.aqi'].diff()

df = pd.merge(df_weather, df_aqi, on='dt', how='inner')
print("Merged rows:", len(df))
print(df.head())


Merged rows: 32
                   dt                                            weather  \
0 2025-09-26 09:00:00  [{'id': 803, 'main': 'Clouds', 'description': ...   
1 2025-09-26 12:00:00  [{'id': 803, 'main': 'Clouds', 'description': ...   
2 2025-09-26 15:00:00  [{'id': 802, 'main': 'Clouds', 'description': ...   
3 2025-09-26 18:00:00  [{'id': 801, 'main': 'Clouds', 'description': ...   
4 2025-09-26 21:00:00  [{'id': 800, 'main': 'Clear', 'description': '...   

   visibility  pop               dt_txt  main.temp  main.feels_like  \
0       10000    0  2025-09-26 09:00:00     305.22           308.94   
1       10000    0  2025-09-26 12:00:00     304.19           308.08   
2       10000    0  2025-09-26 15:00:00     302.38           306.19   
3       10000    0  2025-09-26 18:00:00     300.33           303.19   
4       10000    0  2025-09-26 21:00:00     299.73           299.73   

   main.temp_min  main.temp_max  main.pressure  ...  sys.pod  main.aqi  \
0         302.73         3